In [1]:
cd C:\Users\akkro\source\repos\CPhysProj1 - TimeToScreen\CPhysProj1

C:\Users\akkro\source\repos\CPhysProj1 - TimeToScreen\CPhysProj1


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import imageio
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.animation as animation
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Patch
import colorcet as cc
from mpl_toolkits.mplot3d import Axes3D
import cmocean
from matplotlib.colors import Normalize
import imageio.v2 as imageio

In [3]:
data1 = np.loadtxt('pixelIntensities.csv', delimiter=',', usecols=range(100))
dataT = np.loadtxt('times.csv', delimiter=',', usecols=range(100))

In [4]:
def zero_corresponding_dataT(data, dataT):
    rows, cols = data.shape
    min_value = np.min(dataT)
    for i in range(rows):
        for j in range(cols):
            if data[i, j] == 0:
                dataT[i, j] = min_value
    return dataT

# transforming the time data so that it gives the display frames
dataT = zero_corresponding_dataT(data1, dataT)
total_frames = 200
min_time = np.min(dataT)
max_time = np.max(dataT)
scaled_dataT = ((dataT - min_time) / (max_time - min_time)) * (total_frames - 1)
scaled_dataT = scaled_dataT.astype(int)

In [6]:
# load ray trajectory x,y,z and end values
def load_xyz_points(directory_path, file_template, indices):
    x_coords, y_coords, z_coords = [], [], []
    ray_endpoints = []
    for index in indices:
        filename = file_template.format(index[0], index[1])
        file_path = os.path.join(directory_path, filename)  # Ensure the correct file path
        if os.path.exists(file_path):
            data = np.loadtxt(file_path, delimiter=',', usecols=range(3))
            x_coords.extend(data[:, 0])
            y_coords.extend(data[:, 1])
            z_coords.extend(data[:, 2])
            ray_endpoints.append((data[-1, 0], data[-1, 1], data[-1, 2]))  # Append the last point of each ray
    return x_coords, y_coords, z_coords, ray_endpoints

# create combined plot
def create_combined_plot(data1, scaled_dataT, directory_path, file_template, target_value, save_path=None):
    DPI = 100
    x_pixels, y_pixels = data1.shape[1], data1.shape[0]
    width_inch = x_pixels / DPI
    height_inch = y_pixels / DPI

    # Normalize data1 to the range [0, 1]
    data1_normalized = (data1 - np.min(data1)) / (np.max(data1) - np.min(data1))

    # Apply colormap
    cmap = cc.cm.CET_L3
    data1_mapped = cmap(data1_normalized)

    # Create data1_copy from data1_mapped
    data1_copy = np.zeros_like(data1_mapped)
    data1_copy[:, :, :3] = data1_mapped[:, :, :3]

    # Alter data1_copy so that pixels corresponding to the shown rays are cyan
    for index, value in np.ndenumerate(scaled_dataT):
        if value == target_value:
            data1_copy[index] = [0.0, 1.0, 1.0, 1.0]  

    # If you want to discard the alpha channel (RGBA to RGB):
    data1_copy = data1_copy[:, :, :3]

    # Increase the figure size
    fig = plt.figure(figsize=(width_inch * 16, height_inch * 8), dpi=DPI)
    gs = plt.GridSpec(1, 2, width_ratios=[1, 1])

    # 2D Plot with CET L3 colormap
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.axis('off')
    ax1.imshow(np.flipud(data1_copy), extent=[-24, 24, -24, 24], origin='lower')

    # 3D Plot
    ax2 = fig.add_subplot(gs[0, 1], projection='3d')
    indices = np.argwhere(scaled_dataT == target_value)
    x_coords, y_coords, z_coords, ray_endpoints = load_xyz_points(directory_path, file_template, indices)

     # Skip frames with no rays
    if not x_coords:
        plt.close(fig)
        return False 

    # Separate the coordinates above and below the XY plane
    x_above, y_above, z_above = [], [], []
    x_below, y_below, z_below = [], [], []
    for x, y, z in zip(x_coords, y_coords, z_coords):
        if z >= 0:
            x_above.append(x)
            y_above.append(y)
            z_above.append(z)
        else:
            x_below.append(x)
            y_below.append(y)
            z_below.append(z)

    # Plot points above and below the XY plane with different colors (helps to clarify 3D trajectories)
    ax2.scatter(x_above, y_above, z_above, c='cyan', marker='.')
    ax2.scatter(x_below, y_below, z_below, c='blue', marker='.')

    # Plot the last points of the rays in red
    if ray_endpoints:
        x_end, y_end, z_end = zip(*ray_endpoints)
        ax2.scatter(x_end, y_end, z_end, c='red', marker='o')

    # Plot settings for 3D
    range_limit = 20.0
    ax2.set_xlim(-range_limit, range_limit)
    ax2.set_ylim(-range_limit, range_limit)
    ax2.set_zlim(-range_limit, range_limit)
    ax2.view_init(elev=30, azim=-90)

    # Plot black sphere at the origin representing black hole
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    radius = 1.5
    x = radius * np.outer(np.cos(u), np.sin(v))
    y = radius * np.outer(np.sin(u), np.sin(v))
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
    ax2.plot_surface(x, y, z, color='k', alpha=0.25)

    # Plot orange transparent circular plane with a hole in the center representing the accretion disk
    theta = np.linspace(0, 2 * np.pi, 100)
    radius_outer = 20
    radius_inner = 5
    x_plane, y_plane = [], []
    for r in np.linspace(radius_inner, radius_outer, 100):
        x_plane.append(r * np.cos(theta))
        y_plane.append(r * np.sin(theta))
    x_plane = np.array(x_plane)
    y_plane = np.array(y_plane)
    z_plane = np.zeros_like(x_plane)
    ax2.plot_surface(x_plane, y_plane, z_plane, color='orange', alpha=0.4, rstride=5, cstride=5)

    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0.1)

    # Frame with rays generated successfully
    if save_path:
        plt.savefig(save_path)
    plt.close(fig)
    return True  

# Usage
file_template = '{0},{1}.csv'
frames = []
output_directory = 'frames'
os.makedirs(output_directory, exist_ok=True)

# Generate frames and save them
directory_path = r'C:\Users\akkro\source\repos\CPhysProj1 - TimeToScreen\CPhysProj1'
for i in range(1, np.max(scaled_dataT)):
    target_value = i
    save_path = os.path.join(output_directory, f'frame_{i:03d}.png')
    if create_combined_plot(data1, scaled_dataT, directory_path, file_template, target_value, save_path):
        frames.append(save_path)

# Create a video from the frames
video_path = r'C:\Users\akkro\source\repos\CPhysProj1 - TimeToScreen\CPhysProj1\video\TEST.mp4'
with imageio.get_writer(video_path, fps=2) as writer:  # 2 fps for 0.5 seconds per frame
    for frame in frames:
        image = imageio.imread(frame)
        writer.append_data(image)

print(f"Video saved as {video_path}")


Video saved as C:\Users\akkro\source\repos\CPhysProj1 - TimeToScreen\CPhysProj1\video\TEST.mp4
